In [7]:
from dotenv import load_dotenv
import os
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TRAVILY_API_KEY = os.getenv("TRAVILY_API_KEY")

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import GoogleGenerativeAI

In [3]:
llm =  GoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY)

In [6]:
loader=PyPDFLoader("knowledgebase.pdf")
docs = loader.load_and_split()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024)
documents = text_splitter.split_documents(docs)

In [8]:
PERSIST_STORAGE = "./storage"

In [12]:
db = Chroma.from_documents(documents=documents, embedding= embeddings,persist_directory=PERSIST_STORAGE)

[Document(page_content='Hotel\nManagement\nAI\nAgent\nProject\nTitle:\nHAAS\n-\nHotel\nAI\nAssistant\nSystem\nObjective:\nBuild\nan\nAI\nAgent\nfor\na\nHotel\nManagement\nSystem\nthat\nwill\nhave\nthe\npower\nto\nanswer\nthe\nqueries\nof\nthe\ncustomers\nabout\nthe\nhotel,\nroom\navailability,\ncontact\nnumber\nand\nthe\nfacilities\nprovided\nby\nthe\nhotel.\nFeatures\nof\nAI\nAgent\nRoom\nAvailability\nCheck:\n-\nAllow\nusers\nto\nget\ninformation\nabout\nthe\nnumber\nof\nrooms\navailable\nin\nthe\nHotel.\nPrice\nListing:\n-\nAllow\nusers\nto\nknow\nthe\nprices\nof\nthe\nhotel\nrooms\nof\ndifferent\ncategories.\nGuest\nServices:\n-\nProvide\ninformation\nabout\nhotel\namenities,\nnearby\nattractions.\n-\nFacilitate\nroom\nservice\nrequests,\nhousekeeping,\nor\nmaintenance\nqueries.\nCheck-in/Check-out:\n-\nSupport\nseamless\ncheck-in\nand\ncheck-out\nprocesses\nand\ntimings\nFeedback\nand\nSupport:\n-\nGather\nfeedback\nfrom\nguests\nand\nhandle\nsupport\ninquiries\nefficiently.\nTech